In [1]:
%load_ext autoreload
%autoreload 2

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [1]:
import numpy as np

!pip install neurolib
from neurolib.models.aln import ALNModel
from neurolib.utils.loadData import Dataset
ds = Dataset("hcp")

import mopet


!rm exploration.h5

model = ALNModel(Cmat = ds.Cmat, Dmat = ds.Dmat)
#model = ALNModel()
model.params.duration = 1*1000

def evalFunction(params):
    model.params = params
    model.run()
    return model.outputs

params = model.params
explore_params = {"mue_ext_mean" : np.linspace(0, 2, 3)}
ex = mopet.Exploration(evalFunction, params, explore_params)

In [2]:
ex.run()

2020-03-12 23:30:48,864	WARNING worker.py:682 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2020-03-12 23:30:48,915	INFO resource_spec.py:212 -- Starting Ray with 2.44 GiB memory available for workers and up to 1.24 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-12 23:30:49,266	WARNING services.py:1080 -- Failed to start the dashboard. The dashboard requires Python 3 as well as 'pip install aiohttp psutil setproctitle grpcio'.


In [16]:
import ppet
import numpy as np

!rm exploration.h5

def evalFunction(params):
    result_float = abs((params["x"] ** 2 + params["x"] ** 2) - 1)
    result_array = np.random.randn(np.random.randint(1, 131), np.random.randint(1, 5000))
    result = {}
    result["float_result"] = result_float
    result["array_result"] = result_array
    return result

params = {"x": 1.3, "y": 2.7, "z": 0.0}
explore_params = {"x": np.linspace(-3, 3, 11), "y": np.linspace(-3, 3, 10)}
    
ex = ppet.Exploration(evalFunction, params, explore_params)

In [154]:

hdf_filename = "test.h5"
!rm test.h5


h5file = tables.open_file(hdf_filename, mode="a", title="ppet explorations")
run_str =  "run" + datetime.datetime.now().strftime("_%Y_%m_%d_%HH_%MM_%SS")
run_group = h5file.create_group("/",run_str)
results_group = h5file.create_group(h5file.root[run_str], "results")

runs = range(3)
for run in runs:
    run = "run_" + str(run)
    run_results_group = h5file.create_group(h5file.root[run_str], str(run))
    for res in range(2):
        res = "result_" + str(res)
        x = np.random.randn(np.random.randint(1, 13), np.random.randint(1, 23))
        ds = h5file.create_array(run_results_group, str(res), obj = x)

print(h5file)
h5file.close()

test.h5 (File) 'ppet explorations'
Last modif.: 'Thu Mar 12 16:45:38 2020'
Object Tree: 
/ (RootGroup) 'ppet explorations'
/run_2020_03_12_16H_45M_38S (Group) ''
/run_2020_03_12_16H_45M_38S/results (Group) ''
/run_2020_03_12_16H_45M_38S/run_0 (Group) ''
/run_2020_03_12_16H_45M_38S/run_0/result_0 (Array()) ''
/run_2020_03_12_16H_45M_38S/run_0/result_1 (Array()) ''
/run_2020_03_12_16H_45M_38S/run_1 (Group) ''
/run_2020_03_12_16H_45M_38S/run_1/result_0 (Array()) ''
/run_2020_03_12_16H_45M_38S/run_1/result_1 (Array()) ''
/run_2020_03_12_16H_45M_38S/run_2 (Group) ''
/run_2020_03_12_16H_45M_38S/run_2/result_0 (Array()) ''
/run_2020_03_12_16H_45M_38S/run_2/result_1 (Array()) ''



In [6]:
import ray
import numpy as np
import copy

class RayTest():
    def __init__(self, func, params):
        ray.shutdown()
        ray.init()
        
        self.func = func
        self.params = copy.deepcopy(params)

    @ray.remote
    def f(self, params):
        r = self.func(params)
        return r
    
    def run(self):
        self.results = {}
        for i in np.linspace(0, 2, 11):
            self.params["mue_ext_mean"] = i
            a = self.f.remote(self, self.params)
            self.results[i] = a

    def get(self):
        for rkey, ray_result in self.results.items():
            rval = ray.get(ray_result)
            print(rval)

In [7]:
from neurolib.models.aln import ALNModel
model = ALNModel()

INFO:root:aln: Model initialized.


In [8]:
#model.run()

In [11]:
from ppet.ppet import RayTest

In [16]:
# class Test():
#     def __init__(self):
#         a = 0
#     def run(self, b):
#         a = b + 123
#         return a
    
def func(params):
    model.run()
    model.params = params
    return model.outputs


r = RayTest(func, model.params)

2020-03-12 21:34:54,369	WARNING worker.py:682 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2020-03-12 21:34:54,419	INFO resource_spec.py:212 -- Starting Ray with 3.56 GiB memory available for workers and up to 1.79 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-12 21:34:54,765	WARNING services.py:1080 -- Failed to start the dashboard. The dashboard requires Python 3 as well as 'pip install aiohttp psutil setproctitle grpcio'.


In [17]:
r.run()

AttributeError: 'RayTest' object has no attribute 'explore_params_list'

In [15]:
r.get()

{'t': array([1.0000e-01, 2.0000e-01, 3.0000e-01, ..., 1.9998e+03, 1.9999e+03,
       2.0000e+03]), 'rates_exc': array([[26.15770656, 24.06801404, 22.20402896, ...,  4.19831954,
         4.11221427,  4.03077797]]), 'rates_inh': array([[63.79426742, 58.30968605, 53.80882255, ...,  9.53222441,
         9.53602151,  9.53730842]])}
{'t': array([1.0000e-01, 2.0000e-01, 3.0000e-01, ..., 1.9998e+03, 1.9999e+03,
       2.0000e+03]), 'rates_exc': array([[26.15770656, 24.06801404, 22.20402896, ...,  4.19831954,
         4.11221427,  4.03077797]]), 'rates_inh': array([[63.79426742, 58.30968605, 53.80882255, ...,  9.53222441,
         9.53602151,  9.53730842]])}
{'t': array([1.0000e-01, 2.0000e-01, 3.0000e-01, ..., 1.9998e+03, 1.9999e+03,
       2.0000e+03]), 'rates_exc': array([[26.15770656, 24.06801404, 22.20402896, ...,  4.19831954,
         4.11221427,  4.03077797]]), 'rates_inh': array([[63.79426742, 58.30968605, 53.80882255, ...,  9.53222441,
         9.53602151,  9.53730842]])}
{'t': array([

In [28]:
for rkey, ray_result in results.items():
    rval = ray.get(ray_result)
    print(rval)

NameError: name 'results' is not defined

In [44]:
ray.get(a)

1

In [48]:
results

{0: ObjectID(55c3b2b635949d81ffffffff010000c801000000),
 1: ObjectID(9fc77bf30b438997ffffffff010000c801000000),
 2: ObjectID(9d28cb176c7f7501ffffffff010000c801000000),
 3: ObjectID(15c675b22d037e3bffffffff010000c801000000),
 4: ObjectID(8603ac10b8c093b3ffffffff010000c801000000),
 5: ObjectID(276fa5c994b5ce43ffffffff010000c801000000),
 6: ObjectID(2ca53902e2591031ffffffff010000c801000000),
 7: ObjectID(876814b884fba94dffffffff010000c801000000),
 8: ObjectID(7dec8564195ad979ffffffff010000c801000000),
 9: ObjectID(0bead116322a6c2bffffffff010000c801000000)}